In [1]:
import requests
from bs4 import BeautifulSoup
import re
import json
import html
import pandas as pd
import datetime
import time
import random
import warnings
import ast
import folium

from tqdm import tqdm

pd.set_option('display.max_columns', None) 
pd.options.mode.chained_assignment = None

In [27]:
df_full = pd.read_csv('full_dataset2_with_types.csv')
df_full.head()

,id,internal_id,title_abbreviation,address,price,price_per_unit_area,size_with_square_meter,construction_year,levels,bedrooms,bathrooms,date,extraction_date,geo_lat,geo_lng,company_title,account_id,ad_group_id,url,dimos,perioxi,type,area,price_per_area,level_list,min_level,level_length,temp_no,temp_date_unit,days_passed,kwd_ypes,enotites
0,815053331,47143151,Διαμέρισμα 54 τ.μ.,Αθήνα (Λυκαβηττός),560.0,10,54 τ.μ.,1970.0,"['-1', '0']",1.0,1.0,πριν από 2 ημέρες,2023-06-01,37.979777,23.742884,NaN,2267566.0,NaN,https://www.xe.gr/property/d/enoikiaseis-katoi...,Αθηναίων,Λυκαβηττός,Διαμέρισμα,54.0,10.370370,"[-1.0, 0.0]",0.0,2,2,ημ,2,9186,ΚΕΝΤΡΙΚΟΣ ΤΟΜΕΑΣ ΑΘΗΝΩΝ
1,815047933,47142491,Διαμέρισμα 45 τ.μ.,Αθήνα (Άνω Πατήσια),420.0,9,45 τ.μ.,1969.0,['1'],1.0,1.0,πριν από 2 ημέρες,2023-06-01,38.017481,23.735684,NaN,1407351.0,459093.0,https://www.xe.gr/property/d/enoikiaseis-katoi...,Αθηναίων,Άνω Πατήσια,Διαμέρισμα,45.0,9.333333,[1.0],1.0,1,2,ημ,2,9186,ΚΕΝΤΡΙΚΟΣ ΤΟΜΕΑΣ ΑΘΗΝΩΝ
2,815017929,47140206,Διαμέρισμα 60 τ.μ.,Αθήνα (Κουντουριώτικα),780.0,13,60 τ.μ.,1975.0,['2'],1.0,1.0,πριν από 2 ημέρες,2023-06-01,37.992291,23.755579,NaN,110841.0,NaN,https://www.xe.gr/property/d/enoikiaseis-katoi...,Αθηναίων,Κουντουριώτικα,Διαμέρισμα,60.0,13.000000,[2.0],2.0,1,2,ημ,2,9186,ΚΕΝΤΡΙΚΟΣ ΤΟΜΕΑΣ ΑΘΗΝΩΝ
3,814952950,47138240,Διαμέρισμα 55 τ.μ.,Αθήνα (Πολύγωνο),600.0,11,55 τ.μ.,1981.0,['1'],1.0,1.0,πριν από 3 ημέρες,2023-06-01,37.999083,23.755417,NaN,1344554.0,NaN,https://www.xe.gr/property/d/enoikiaseis-katoi...,Αθηναίων,Πολύγωνο,Διαμέρισμα,55.0,10.909091,[1.0],1.0,1,3,ημ,3,9186,ΚΕΝΤΡΙΚΟΣ ΤΟΜΕΑΣ ΑΘΗΝΩΝ
4,814956607,47138448,Διαμέρισμα 50 τ.μ.,Αθήνα (Πανόρμου),790.0,16,50 τ.μ.,1970.0,['1'],1.0,1.0,πριν από 3 ημέρες,2023-06-01,37.990937,23.765072,BLUEGROUND REAL ESTATE,1220409.0,NaN,https://www.xe.gr/property/d/enoikiaseis-katoi...,Αθηναίων,Πανόρμου,Διαμέρισμα,50.0,15.800000,[1.0],1.0,1,3,ημ,3,9186,ΚΕΝΤΡΙΚΟΣ ΤΟΜΕΑΣ ΑΘΗΝΩΝ


In [31]:
unit_price = df_full.groupby('dimos').agg({'price_per_area': 'mean', 'kwd_ypes': 'first'}).reset_index()
unit_price

,dimos,price_per_area,kwd_ypes
0,Άγιοι Ανάργυροι - Καματερό,7.179965,9180
1,Άγιος Δημήτριος,10.541667,9194
2,Ίλιον,7.463547,9182
3,Αθηναίων,10.324897,9186
4,Αιγάλεω,7.541798,9181
5,Βύρωνας,9.730185,9187
6,Γαλάτσι,8.487865,9188
7,Γλυφάδα,17.692308,9196
8,Δάφνη - Υμηττός,10.501890,9189
9,Ζωγράφου,9.901282,9190


In [4]:
from folium.features import GeoJsonTooltip
import geopandas as gpd

In [8]:
url = 'http://gis.cityofathens.gr/geoserver/ows?service=WFS&version=1.0.0&request=GetFeature&typename=geonode%3Aper_enot_attki&outputFormat=json&srs=EPSG%3A2100&srsName=EPSG%3A2100'

response = requests.get(url)
if response.status_code == 200:
    with open('dimoi__athens_new.geojson', 'wb') as file:
        file.write(response.content)
    print('GeoJSON file downloaded successfully.')
else:
    print('Failed to download the GeoJSON file.')

GeoJSON file downloaded successfully.


In [9]:
#Read the geoJSON file using geopandas
geojson = gpd.read_file('dimoi__athens_new.geojson')

geojson #only select 'name' and 'geometry' columns

,id,fid_1,gid,fid,name,kwd_ypes,temp,label,enotites,geometry
0,per_enot_attki.1,1,111,1,Ηλιούπολης,9191,0.0,Δήμος Ηλιούπολης,ΚΕΝΤΡΙΚΟΣ ΤΟΜΕΑΣ ΑΘΗΝΩΝ,"MULTIPOLYGON (((478558.812 4199936.000, 478583..."
1,per_enot_attki.2,2,114,2,Ηρακλείου,9170,0.0,Δήμος Ηρακλείου,ΒΟΡΕΙΟΥ ΤΟΜΕΑ ΑΘΗΝΩΝ,"MULTIPOLYGON (((480042.312 4212790.000, 480048..."
2,per_enot_attki.3,3,181,3,Λυκόβρυσης - Πεύκης,9172,0.0,Δήμος Λυκόβρυσης - Πεύκης,ΒΟΡΕΙΟΥ ΤΟΜΕΑ ΑΘΗΝΩΝ,"MULTIPOLYGON (((481198.812 4214116.000, 481244..."
3,per_enot_attki.4,4,18,4,Αγίων Αναργύρων - Καματερού,9180,0.0,Δήμος Αγίων Αναργύρων - Καματερού,ΔΥΤΙΚΟΥ ΤΟΜΕΑ ΑΘΗΝΩΝ,"MULTIPOLYGON (((473922.000 4212995.000, 474629..."
4,per_enot_attki.5,5,25,5,Αιγάλεω,9181,1.0,Δήμος Αιγάλεω,ΔΥΤΙΚΟΥ ΤΟΜΕΑ ΑΘΗΝΩΝ,"MULTIPOLYGON (((470978.594 4206591.000, 470997..."
...,...,...,...,...,...,...,...,...,...,...
59,per_enot_attki.60,60,248,60,Περάματος,9206,0.0,Δήμος Περάματος,ΠΕΙΡΑΙΩΣ,"MULTIPOLYGON (((461242.594 4204347.000, 461235..."
60,per_enot_attki.61,61,250,61,Πετρούπολης,9184,0.0,Δήμος Πετρούπολης,ΔΥΤΙΚΟΥ ΤΟΜΕΑ ΑΘΗΝΩΝ,"MULTIPOLYGON (((471709.906 4212317.000, 471922..."
61,per_enot_attki.62,62,259,62,Πόρου,9210,NaN,Δήμος Πόρου,ΝΗΣΩΝ,"MULTIPOLYGON (((452989.812 4149075.000, 452987..."
62,per_enot_attki.63,63,265,64,Ραφήνας - Πικερμίου,9224,0.0,Δήμος Ραφήνας - Πικερμίου,ΑΝΑΤΟΛΙΚΗΣ ΑΤΤΙΚΗΣ,"MULTIPOLYGON (((491820.312 4209970.000, 491877..."


In [18]:
geojson_clean = geojson[~geojson['enotites'].isin(['ΠΕΙΡΑΙΩΣ','ΝΗΣΩΝ','ΑΝΑΤΟΛΙΚΗΣ ΑΤΤΙΚΗΣ','ΔΥΤΙΚΗΣ ΑΤΤΙΚΗΣ'])]

In [19]:
geojson_clean2 = geojson_clean['kwd_ypes','geometry']

,id,fid_1,gid,fid,name,kwd_ypes,temp,label,enotites,geometry
0,per_enot_attki.1,1,111,1,Ηλιούπολης,9191,0.0,Δήμος Ηλιούπολης,ΚΕΝΤΡΙΚΟΣ ΤΟΜΕΑΣ ΑΘΗΝΩΝ,"MULTIPOLYGON (((478558.812 4199936.000, 478583..."
1,per_enot_attki.2,2,114,2,Ηρακλείου,9170,0.0,Δήμος Ηρακλείου,ΒΟΡΕΙΟΥ ΤΟΜΕΑ ΑΘΗΝΩΝ,"MULTIPOLYGON (((480042.312 4212790.000, 480048..."
2,per_enot_attki.3,3,181,3,Λυκόβρυσης - Πεύκης,9172,0.0,Δήμος Λυκόβρυσης - Πεύκης,ΒΟΡΕΙΟΥ ΤΟΜΕΑ ΑΘΗΝΩΝ,"MULTIPOLYGON (((481198.812 4214116.000, 481244..."
3,per_enot_attki.4,4,18,4,Αγίων Αναργύρων - Καματερού,9180,0.0,Δήμος Αγίων Αναργύρων - Καματερού,ΔΥΤΙΚΟΥ ΤΟΜΕΑ ΑΘΗΝΩΝ,"MULTIPOLYGON (((473922.000 4212995.000, 474629..."
4,per_enot_attki.5,5,25,5,Αιγάλεω,9181,1.0,Δήμος Αιγάλεω,ΔΥΤΙΚΟΥ ΤΟΜΕΑ ΑΘΗΝΩΝ,"MULTIPOLYGON (((470978.594 4206591.000, 470997..."
7,per_enot_attki.8,8,27,8,Αλίμου,9195,0.0,Δήμος Αλίμου,ΝΟΤΙΟΣ ΤΟΜΕΑΣ ΑΘΗΝΩΝ,"MULTIPOLYGON (((477383.812 4197610.000, 477418..."
9,per_enot_attki.10,10,208,10,Νέας Ιωνίας,9174,0.0,Δήμος Νέας Ιωνίας,ΒΟΡΕΙΟΥ ΤΟΜΕΑ ΑΘΗΝΩΝ,"MULTIPOLYGON (((477996.188 4211176.000, 478008..."
11,per_enot_attki.12,12,238,12,Παλαιού Φαλήρου,9201,0.0,Δήμος Παλαιού Φαλήρου,ΝΟΤΙΟΣ ΤΟΜΕΑΣ ΑΘΗΝΩΝ,"MULTIPOLYGON (((473476.812 4199280.000, 473643..."
12,per_enot_attki.13,13,76,17,Δάφνης - Υμηττού,9189,0.0,Δήμος Δάφνης - Υμηττού,ΚΕΝΤΡΙΚΟΣ ΤΟΜΕΑΣ ΑΘΗΝΩΝ,"MULTIPOLYGON (((477683.821 4200833.448, 477837..."
15,per_enot_attki.16,16,242,14,Παπάγου - Χολαργού,9175,0.0,Δήμος Παπάγου - Χολαργού,ΒΟΡΕΙΟΥ ΤΟΜΕΑ ΑΘΗΝΩΝ,"MULTIPOLYGON (((483179.594 4205708.000, 483216..."


In [22]:
geojson_clean2 = geojson_clean[["kwd_ypes","geometry"]]

In [33]:
geojson_clean2.to_file('geojson_clean2.geojson', driver='GeoJSON')

In [26]:
geojson.to_csv('athens_jeo.csv', encoding='utf-8-sig')

In [ ]:
#Create the choropleth map add it to the base map
#custom_scale = (df_final['new_cases_7days'].quantile((0,0.2,0.4,0.6,0.8,1))).tolist()
folium.Choropleth(
            geo_data=r'geojson_clean2',
            data=unit_price,
            columns=['kwd_ypes', 'price_per_area'],  #Here we tell folium to get the county fips and plot new_cases_7days metric for each county
            key_on='feature.properties.coty_code', #Here we grab the geometries/county boundaries from the geojson file using the key 'coty_code' which is the same as county fips
            #threshold_scale=custom_scale, #use the custom scale we created for legend
            fill_color='YlOrRd',
            nan_fill_color="White", #Use white color if there is no data available for the county
            fill_opacity=0.7,
            line_opacity=0.2,
            legend_name='New Cases Past 7 Days (Per 100K Population) ', #title of the legend
            highlight=True,
            line_color='black').add_to(us_map) 

us_map

In [36]:
# Create a folium map centered on Athens
athens_map = folium.Map(location=[37.9838, 23.7275], zoom_start=12)

# Add the choropleth layer to the map
folium.Choropleth(
    geo_data='geojson_clean2.geojson',
    name='choropleth',
    data=unit_price,
    columns=['kwd_ypes', 'price_per_area'],
    key_on='feature.properties.kwd_ypes',
    fill_color='YlOrRd',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Price per unit area',
    highlight=True,
    line_color='black'
).add_to(athens_map)

# Add a layer control to the map
folium.LayerControl().add_to(athens_map)

athens_map

# Save the map as an HTML file
#athens_map.save('athens_choropleth_map.html')